In [1]:
# check the model of using differnet window size, compare their results on valid dataset
#    test dataset without any preprocessing, no scaling and no normalization
# decide which window size is the optimal

In [2]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
%matplotlib qt


import torchvision.datasets as dsets
import torchvision.transforms as transforms
# from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader
from torch.autograd import Variable

import my_model
from utilities import MyTrainDataSet, MyTestDataSet, load_data_2, load_test_data, min_max_scaling, normalize_one, construct_train_valid_tensor, construct_test_tensor, show_statistic

x mean: 0.5009409709733069 ; std: 0.2436898615684393
y mean: 0.4416590158205868 ; std: 0.23351418998386245
z mean: 0.48521343842977466 ; std: 0.2423616199566707

In [3]:
# mean and std from training data set
x_mean = 0.5009409709733069
x_std = 0.2436898615684393
y_mean = 0.4416590158205868
y_std = 0.23351418998386245
z_mean = 0.48521343842977466
z_std = 0.2423616199566707

In [4]:
x_mean

0.5009409709733069

Plot result with normalization

In [5]:
test_1_loss = []
for i in range(1, 11):
    window_size = i
    PATH = "model/with_early_stop_after_400/model_w_" + str(i) + ".pt"

    test_set_x_1, test_set_y_1, test_set_z_1 = load_test_data('../../../../performance_test/data/test/test_1.csv')
    test_set_x_2, test_set_y_2, test_set_z_2 = load_test_data('../../../../performance_test/data/test/test_2.csv')
    test_set_x_3, test_set_y_3, test_set_z_3 = load_test_data('../../../../performance_test/data/test/test_3.csv')
    test_set_x_4, test_set_y_4, test_set_z_4 = load_test_data('../../../../performance_test/data/test/test_4.csv')
    test_set_x_5, test_set_y_5, test_set_z_5 = load_test_data('../../../../performance_test/data/test/test_5.csv')

    # do min-max-scaling for each test data set
    show_statistic(test_set_x_1)
    min_max_scaling(test_set_x_1)
    show_statistic(test_set_x_1)
    min_max_scaling(test_set_x_2)
    min_max_scaling(test_set_x_3)
    min_max_scaling(test_set_x_4)
    min_max_scaling(test_set_x_5)

    min_max_scaling(test_set_y_1)
    min_max_scaling(test_set_y_2)
    min_max_scaling(test_set_y_3)
    min_max_scaling(test_set_y_4)
    min_max_scaling(test_set_y_5)

    min_max_scaling(test_set_z_1)
    min_max_scaling(test_set_z_2)
    min_max_scaling(test_set_z_3)
    min_max_scaling(test_set_z_4)
    min_max_scaling(test_set_z_5)


    show_statistic(test_set_x_1)
    # do normalization on x of validation test set
    normalize_one(test_set_x_1, x_mean, x_std)
    show_statistic(test_set_x_1)
    normalize_one(test_set_x_2, x_mean, x_std)
    normalize_one(test_set_x_3, x_mean, x_std)
    normalize_one(test_set_x_4, x_mean, x_std)
    normalize_one(test_set_x_5, x_mean, x_std)
    # do normalization on y of validation test set
    normalize_one(test_set_y_1, y_mean, y_std)
    normalize_one(test_set_y_2, y_mean, y_std)
    normalize_one(test_set_y_3, y_mean, y_std)
    normalize_one(test_set_y_4, y_mean, y_std)
    normalize_one(test_set_y_5, y_mean, y_std)
    # do normalization on z of validation test set
    normalize_one(test_set_z_1, z_mean, z_std)
    normalize_one(test_set_z_2, z_mean, z_std)
    normalize_one(test_set_z_3, z_mean, z_std)
    normalize_one(test_set_z_4, z_mean, z_std)
    normalize_one(test_set_z_5, z_mean, z_std)
    # show_statistic(train_set_x_1)


    test_dataset_1, test_label_1 = construct_test_tensor(test_set_x_1,
                                                         test_set_y_1,
                                                         test_set_z_1,
                                                         window_size)
    test_dataset_2, test_label_2 = construct_test_tensor(test_set_x_2,
                                                         test_set_y_2,
                                                         test_set_z_2,
                                                         window_size)
    test_dataset_3, test_label_3 = construct_test_tensor(test_set_x_3,
                                                         test_set_y_3,
                                                         test_set_z_3,
                                                         window_size)
    test_dataset_4, test_label_4 = construct_test_tensor(test_set_x_4,
                                                         test_set_y_4,
                                                         test_set_z_4,
                                                         window_size)
    test_dataset_5, test_label_5 = construct_test_tensor(test_set_x_5,
                                                         test_set_y_5,
                                                         test_set_z_5,
                                                         window_size)

    test_set_1 = MyTestDataSet(test_dataset_1, test_label_1)
    test_set_2 = MyTestDataSet(test_dataset_2, test_label_2)
    test_set_3 = MyTestDataSet(test_dataset_3, test_label_3)
    test_set_4 = MyTestDataSet(test_dataset_4, test_label_4)
    test_set_5 = MyTestDataSet(test_dataset_5, test_label_5)
    print(len(test_set_1))
    print(len(test_set_2))
    print(len(test_set_3))
    print(len(test_set_4))
    print(len(test_set_5))
    show_statistic(test_set_x_1)
    show_statistic(test_set_x_2)
    show_statistic(test_set_x_3)
    show_statistic(test_set_x_4)
    show_statistic(test_set_x_5)


    batch_size = 650
    test_loader_1 = DataLoader(test_set_1, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_2 = DataLoader(test_set_2, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_3 = DataLoader(test_set_3, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_4 = DataLoader(test_set_4, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_5 = DataLoader(test_set_5, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    
    input_dim = 3
    hidden_dim = 100
    layer_dim = 1
    output_dim = 3

    load_model = my_model.LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
    load_model.load_state_dict(torch.load(PATH))
    load_model.eval()
    # mmodel = torch.load(PATH)

    criterion = nn.MSELoss()

    if torch.cuda.is_available():
        load_model.cuda()

    # get test results
    seq_dim = window_size
    input_dim = 3
    # test_seq = []
    test_predd = []
    # test_gt = []
    total_test_loss = 0.0
    test_batch = 0
    for i, (seqs, labels) in enumerate(test_loader_1):
        if torch.cuda.is_available():
            seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            seqs = Variable(seqs.view(-1, seq_dim, input_dim))

        outputs = load_model(seqs)
        loss = criterion(outputs, labels)
        total_test_loss += loss.data.item()
        test_predd.append(outputs)
        test_batch = i + 1

    print(total_test_loss/test_batch)
    test_1_loss.append(total_test_loss/test_batch)

    for i in range(len(test_predd)):
        if (i == 0):
            pred = test_predd[i].cpu().detach().numpy()
        else:
            pred = np.append(pred, test_predd[i].cpu().detach().numpy(), axis = 0)

    from mpl_toolkits import mplot3d
    fig = plt.figure()
    ax = plt.axes(projection='3d')

    # Data for a three-dimensional line
    zline = np.linspace(0, 15, 1000)
    xline = np.sin(zline)
    yline = np.cos(zline)
    ax.plot3D(test_set_x_1, test_set_y_1, test_set_z_1, 'gray')
    ax.plot3D(pred[:,0], pred[:,1], pred[:,2], 'red')

-----------------------------
min: -1.5190000000000001
max: 1.41144
mean: -0.058533702725000004
std: 0.7861266362964188
-----------------------------
min: 0.0
max: 1.0
mean: 0.4983778194656777
std: 0.26826232111779097
-----------------------------
min: 0.0
max: 1.0
mean: 0.4983778194656777
std: 0.26826232111779097
-----------------------------
min: -2.0556496185321183
max: 2.0479269257023827
mean: -0.010518088406027203
std: 1.1008349686408698
399
399
399
399
399
-----------------------------
min: -2.0556496185321183
max: 2.0479269257023827
mean: -0.010518088406027203
std: 1.1008349686408698
-----------------------------
min: -2.0556496185321183
max: 2.0479269257023827
mean: -0.003910799069243863
std: 1.3327171046033424
-----------------------------
min: -2.0556496185321183
max: 2.0479269257023827
mean: -0.02591810651939787
std: 1.1851356406826778
-----------------------------
min: -2.0556496185321183
max: 2.0479269257023827
mean: 0.03763954942048355
std: 1.0437368311448298
------------

-----------------------------
min: -1.5190000000000001
max: 1.41144
mean: -0.058533702725000004
std: 0.7861266362964188
-----------------------------
min: 0.0
max: 1.0
mean: 0.4983778194656777
std: 0.26826232111779097
-----------------------------
min: 0.0
max: 1.0
mean: 0.4983778194656777
std: 0.26826232111779097
-----------------------------
min: -2.0556496185321183
max: 2.0479269257023827
mean: -0.010518088406027203
std: 1.1008349686408698
391
391
391
391
391
-----------------------------
min: -2.0556496185321183
max: 2.0479269257023827
mean: -0.010518088406027203
std: 1.1008349686408698
-----------------------------
min: -2.0556496185321183
max: 2.0479269257023827
mean: -0.003910799069243863
std: 1.3327171046033424
-----------------------------
min: -2.0556496185321183
max: 2.0479269257023827
mean: -0.02591810651939787
std: 1.1851356406826778
-----------------------------
min: -2.0556496185321183
max: 2.0479269257023827
mean: 0.03763954942048355
std: 1.0437368311448298
------------

In [ ]:
test_1_loss

In [ ]:
#find window size 6 is optimal

In [ ]:
x = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

x_pos = [i for i, _ in enumerate(x)]

# plt.bar(x_pos, test_1_loss)
plt.bar(x_pos, test_1_loss, color=['tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:green',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue'], zorder = 3)

plt.grid(zorder=0)

plt.xlabel("Input window size", fontsize=14)
plt.ylabel("MSE loss", fontsize=14)
plt.xticks(x_pos, x)

plt.ylim([0.017,0.0255])
# plt.show()

Plot result without normalization

In [6]:
test_3_loss = []
for i in range(1, 11):
    window_size = i
    PATH = "model_v3/with_early_stop_after_400_without_normalization/model_w_" + str(i) + ".pt"

    test_set_x_1, test_set_y_1, test_set_z_1 = load_test_data('../../../../performance_test/data/test/test_1.csv')
    test_set_x_2, test_set_y_2, test_set_z_2 = load_test_data('../../../../performance_test/data/test/test_2.csv')
    test_set_x_3, test_set_y_3, test_set_z_3 = load_test_data('../../../../performance_test/data/test/test_3.csv')
    test_set_x_4, test_set_y_4, test_set_z_4 = load_test_data('../../../../performance_test/data/test/test_4.csv')
    test_set_x_5, test_set_y_5, test_set_z_5 = load_test_data('../../../../performance_test/data/test/test_5.csv')

    '''
    # do min-max-scaling for each test data set
    show_statistic(test_set_x_1)
    min_max_scaling(test_set_x_1)
    show_statistic(test_set_x_1)
    min_max_scaling(test_set_x_2)
    min_max_scaling(test_set_x_3)
    min_max_scaling(test_set_x_4)
    min_max_scaling(test_set_x_5)

    min_max_scaling(test_set_y_1)
    min_max_scaling(test_set_y_2)
    min_max_scaling(test_set_y_3)
    min_max_scaling(test_set_y_4)
    min_max_scaling(test_set_y_5)

    min_max_scaling(test_set_z_1)
    min_max_scaling(test_set_z_2)
    min_max_scaling(test_set_z_3)
    min_max_scaling(test_set_z_4)
    min_max_scaling(test_set_z_5)
    '''
    '''
    show_statistic(test_set_x_1)
    # do normalization on x of validation test set
    normalize_one(test_set_x_1, x_mean, x_std)
    show_statistic(test_set_x_1)
    normalize_one(test_set_x_2, x_mean, x_std)
    normalize_one(test_set_x_3, x_mean, x_std)
    normalize_one(test_set_x_4, x_mean, x_std)
    normalize_one(test_set_x_5, x_mean, x_std)
    # do normalization on y of validation test set
    normalize_one(test_set_y_1, y_mean, y_std)
    normalize_one(test_set_y_2, y_mean, y_std)
    normalize_one(test_set_y_3, y_mean, y_std)
    normalize_one(test_set_y_4, y_mean, y_std)
    normalize_one(test_set_y_5, y_mean, y_std)
    # do normalization on z of validation test set
    normalize_one(test_set_z_1, z_mean, z_std)
    normalize_one(test_set_z_2, z_mean, z_std)
    normalize_one(test_set_z_3, z_mean, z_std)
    normalize_one(test_set_z_4, z_mean, z_std)
    normalize_one(test_set_z_5, z_mean, z_std)
    # show_statistic(train_set_x_1)
    '''

    test_dataset_1, test_label_1 = construct_test_tensor(test_set_x_1,
                                                         test_set_y_1,
                                                         test_set_z_1,
                                                         window_size)
    test_dataset_2, test_label_2 = construct_test_tensor(test_set_x_2,
                                                         test_set_y_2,
                                                         test_set_z_2,
                                                         window_size)
    test_dataset_3, test_label_3 = construct_test_tensor(test_set_x_3,
                                                         test_set_y_3,
                                                         test_set_z_3,
                                                         window_size)
    test_dataset_4, test_label_4 = construct_test_tensor(test_set_x_4,
                                                         test_set_y_4,
                                                         test_set_z_4,
                                                         window_size)
    test_dataset_5, test_label_5 = construct_test_tensor(test_set_x_5,
                                                         test_set_y_5,
                                                         test_set_z_5,
                                                         window_size)

    test_set_1 = MyTestDataSet(test_dataset_1, test_label_1)
    test_set_2 = MyTestDataSet(test_dataset_2, test_label_2)
    test_set_3 = MyTestDataSet(test_dataset_3, test_label_3)
    test_set_4 = MyTestDataSet(test_dataset_4, test_label_4)
    test_set_5 = MyTestDataSet(test_dataset_5, test_label_5)
    print(len(test_set_1))
    print(len(test_set_2))
    print(len(test_set_3))
    print(len(test_set_4))
    print(len(test_set_5))
    show_statistic(test_set_x_1)
    show_statistic(test_set_x_2)
    show_statistic(test_set_x_3)
    show_statistic(test_set_x_4)
    show_statistic(test_set_x_5)


    batch_size = 650
    test_loader_1 = DataLoader(test_set_1, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_2 = DataLoader(test_set_2, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_3 = DataLoader(test_set_3, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_4 = DataLoader(test_set_4, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_5 = DataLoader(test_set_5, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    
    input_dim = 3
    hidden_dim = 100
    layer_dim = 1
    output_dim = 3

    load_model = my_model.LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
    load_model.load_state_dict(torch.load(PATH))
    load_model.eval()
    # mmodel = torch.load(PATH)

    criterion = nn.MSELoss()

    if torch.cuda.is_available():
        load_model.cuda()

    # get test results
    seq_dim = window_size
    input_dim = 3
    # test_seq = []
    test_predd = []
    # test_gt = []
    total_test_loss = 0.0
    test_batch = 0
    for i, (seqs, labels) in enumerate(test_loader_3):
        if torch.cuda.is_available():
            seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            seqs = Variable(seqs.view(-1, seq_dim, input_dim))

        outputs = load_model(seqs)
        loss = criterion(outputs, labels)
        total_test_loss += loss.data.item()
        test_predd.append(outputs)
        test_batch = i + 1

    print(total_test_loss/test_batch)
    test_3_loss.append(total_test_loss/test_batch)

    for i in range(len(test_predd)):
        if (i == 0):
            pred = test_predd[i].cpu().detach().numpy()
        else:
            pred = np.append(pred, test_predd[i].cpu().detach().numpy(), axis = 0)

    from mpl_toolkits import mplot3d
    fig = plt.figure()
    ax = plt.axes(projection='3d')

    # Data for a three-dimensional line
    zline = np.linspace(0, 15, 1000)
    xline = np.sin(zline)
    yline = np.cos(zline)
    ax.plot3D(test_set_x_3, test_set_y_3, test_set_z_3, 'gray')
    ax.plot3D(pred[:,0], pred[:,1], pred[:,2], 'red')

399
399
399
399
399
-----------------------------
min: -1.5190000000000001
max: 1.41144
mean: -0.058533702725000004
std: 0.7861266362964188
-----------------------------
min: -1.9802
max: 1.8685200000000002
mean: -0.05588638135000003
std: 1.2499474347653021
-----------------------------
min: -1.99921
max: 1.99613
mean: -0.023014987725000068
std: 1.153876327053727
-----------------------------
min: -1.9872900000000002
max: 1.98968
mean: 0.04141547985
std: 1.0115346992101457
-----------------------------
min: -1.90428
max: 1.99623
mean: -0.011767208575000021
std: 1.0308403247874398
0.03818032145500183
398
398
398
398
398
-----------------------------
min: -1.5190000000000001
max: 1.41144
mean: -0.058533702725000004
std: 0.7861266362964188
-----------------------------
min: -1.9802
max: 1.8685200000000002
mean: -0.05588638135000003
std: 1.2499474347653021
-----------------------------
min: -1.99921
max: 1.99613
mean: -0.023014987725000068
std: 1.153876327053727
---------------------------

In [7]:
test_3_loss

[0.03818032145500183,
 0.02740023471415043,
 0.026449186727404594,
 0.026836641132831573,
 0.026785902678966522,
 0.02710924670100212,
 0.027639009058475494,
 0.02746979147195816,
 0.027946196496486664,
 0.028636211529374123]

In [8]:
min(test_3_loss)

0.026449186727404594

In [17]:
x = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

x_pos = [i for i, _ in enumerate(x)]


plt.figure(figsize=(6, 5))
# plt.bar(x_pos, test_1_loss)
plt.bar(x_pos, test_3_loss, color=['tab:blue',
                                   'tab:blue',
                                   'tab:green',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue'], zorder = 3)

plt.grid(zorder=0)

plt.xlabel("Input Window Size", fontsize=15)
plt.ylabel("MSE Loss", fontsize=15)
plt.xticks(x_pos, x)

# plt.ylim([0.017,0.0255])
plt.ylim([0.025,0.039])
# plt.show()

plt.tight_layout()

plt.savefig("window_size_tuning.png", format="png", dpi=300)

In [20]:
x = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

# x_pos = [i for i, _ in enumerate(x)]


plt.figure(figsize=(10, 4))
# plt.figure()
plt.plot(x, test_3_loss, c='tab:blue', marker='o')
plt.scatter(x[2], test_3_loss[2], marker='s', s=250, c='tab:green')
plt.grid(zorder=0)






plt.xlabel("Input Window Size", fontsize=15)
plt.ylabel("MSE Loss", fontsize=15)
# plt.ylim([-5.75, -4.0])
# plt.xticks(x_pos, x)

Text(0, 0.5, 'MSE Loss')